In [10]:
import sys
sys.path.append('src')

In [11]:
from bm25_v2 import BM25Searcher
import pandas as pd
import numpy as np
from utils import get_combined_df, Args
from eval import SearchEvaluator, ModelEvaluator

In [12]:
data_path="data/2_7/facebook_react"
repo_name = data_path.split("/")[-1]
combined_df = get_combined_df(data_path)
index_path=f"{data_path}/index_commit_tokenized"
k=1000 # initial ranker depth
n=100 # number of samples to evaluate on

In [15]:
metrics = ['MAP', 'P@1', 'P@10', 'P@20', 'P@30', 'MRR', 'R@1', 'R@10', 'R@100', 'R@1000']
bm25_searcher = BM25Searcher(index_path)
evaluator = SearchEvaluator(metrics)
model_evaluator = ModelEvaluator(bm25_searcher, evaluator, combined_df)
gold_df = pd.read_parquet(f"gold/{repo_name}/v2_{repo_name}_gpt4_gold.parquet")
#! rename commit_message to original_message
gold_df = gold_df.rename(columns={'commit_message': 'original_message'})
#!rename transformed_message to commit_message
gold_df = gold_df.rename(columns={f'transformed_message_gpt4': 'commit_message'})

Loaded index at data/2_7/facebook_react/index_commit_tokenized
Index Stats: {'total_terms': 7587973, 'documents': 73765, 'non_empty_documents': 73765, 'unique_terms': 14602}


In [18]:
test_id, test_date, test_orig, test_files, test_query = gold_df.iloc[6]
test_query, test_files

("Input properties 'value' and 'defaultValue' accepting and assigning functions and symbols leads to improper handling and inconsistencies in numeric equality checks.",
 array(['packages/react-dom/src/__tests__/ReactDOMInput-test.js',
        'packages/react-dom/src/client/ReactDOMFiberInput.js',
        'packages/react-dom/src/events/ChangeEventPlugin.js',
        'packages/react-dom/src/shared/DOMProperty.js'], dtype=object))

In [19]:
bm25_results = bm25_searcher.search(test_query, test_date, k)

In [20]:
aggregated_results = bm25_searcher.aggregate_file_scores(bm25_results, 'sump')

In [22]:
bm25_results

[SearchResult(score=29.04990, file_path='fixtures/dom/src/components/fixtures/number-inputs/index.js', commit_id='fd69c239a0bb8fde412aa7c1c0cea9c7fda287a9', commit_date=1512077095),
 SearchResult(score=29.04990, file_path='packages/react-dom/src/__tests__/ReactDOMInput-test.js', commit_id='fd69c239a0bb8fde412aa7c1c0cea9c7fda287a9', commit_date=1512077095),
 SearchResult(score=29.04990, file_path='packages/react-dom/src/client/DOMPropertyOperations.js', commit_id='fd69c239a0bb8fde412aa7c1c0cea9c7fda287a9', commit_date=1512077095),
 SearchResult(score=29.04990, file_path='packages/react-dom/src/client/ReactDOMFiberInput.js', commit_id='fd69c239a0bb8fde412aa7c1c0cea9c7fda287a9', commit_date=1512077095),
 SearchResult(score=29.04990, file_path='packages/react-dom/src/shared/DOMProperty.js', commit_id='fd69c239a0bb8fde412aa7c1c0cea9c7fda287a9', commit_date=1512077095),
 SearchResult(score=29.04989, file_path='packages/react-dom/src/shared/HTMLDOMPropertyConfig.js', commit_id='fd69c239a0bb8f

In [10]:
aggregated_results[0]

AggregatedSearchResult(file_path='fixtures/dom/src/components/fixtures/number-inputs/index.js', score=66.70698738098145, contributing_results=[SearchResult(score=29.04990, file_path='fixtures/dom/src/components/fixtures/number-inputs/index.js', commit_id='fd69c239a0bb8fde412aa7c1c0cea9c7fda287a9', commit_date=1512077095), SearchResult(score=24.52640, file_path='fixtures/dom/src/components/fixtures/number-inputs/index.js', commit_id='29d9710892f773b4d5081f1b77820d40d79f69d3', commit_date=1490632758), SearchResult(score=13.13069, file_path='fixtures/dom/src/components/fixtures/number-inputs/index.js', commit_id='2beec2f308e27dbe902a8d68e1fc9f3d70ee9372', commit_date=1491946083)])

In [11]:
sump_results = model_evaluator.evaluate_df(gold_df, 1000, aggregation_strategy='sump', rerankers=[], output_folder_path='/home/ssg2/ssg2/ds/out/facebook_react/bm25/debug')

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:22<00:00,  4.52it/s]


In [12]:
get_avg_metrics(sump_results)

NameError: name 'get_avg_metrics' is not defined

In [ ]:
bm25_output_path = 'out/facebook_react/bm25/debug'

In [ ]:
from datetime import date
from utils import AggregatedSearchResult, get_avg_metrics
import os
import json
from typing import List, Dict

# Function to generate .teIn file
def generate_teIn_file(commit_results: Dict[str, List[AggregatedSearchResult]], folder_path: str):
    os.makedirs(folder_path, exist_ok=True)
    file_path = os.path.join(folder_path, "output.teIn")

    with open(file_path, "w") as file:
        for commit_id, results in commit_results.items():
            for result in results:
                file.write(f'{commit_id}\t{result.file_path}\t{result.score}\t{[{"score": sr.score, "file_path": sr.file_path, "commit_id": sr.commit_id, "commit_date": sr.commit_date} for sr in result.contributing_results]}\n')

# Function to generate .qry file
def generate_qry_file(gold_df, folder_path: str, use_gpt_train: bool = True):
    # Ensure folder exists
    os.makedirs(folder_path, exist_ok=True)
    file_path = os.path.join(folder_path, "output.qry")
    with open(file_path, "w") as file:
        for index, row in gold_df.iterrows():
            cid, date, orig, files, query = row
            for f in files:
                file.write(f'{cid}\t{date}\t{f}\n')

            # write in order cid, date, files, query, orig
            # file.write(f'{cid}\t{[f"{f}" for f in files]}\n')
            # file.write(f"{row['commit_id']}\t{msg}\n")

In [ ]:
test_query

"Input properties 'value' and 'defaultValue' accepting and assigning functions and symbols leads to improper handling and inconsistencies in numeric equality checks."

In [ ]:
commit_results = {cid : bm25_searcher.pipeline(query, date, k, 'sump') for cid, date, _, _, query in gold_df.itertuples(index=False, name=None)}

In [ ]:
generate_teIn_file(commit_results, bm25_output_path)

In [ ]:
generate_qry_file(gold_df, bm25_output_path)

In [ ]:
def evaluate_results(teIn_path, qry_path, output_folder):
    # Read actual files modified from .qry file
    actual_files = {}
    total_list = []
    with open(qry_path, 'r') as f:
        for line in f:
            parts = line.strip().split('\t')
            cid = parts[0]
            files_str = parts[1].replace("'", '"')
            try:
                files = json.loads(files_str)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON for commit {cid}: {e}")
                continue  # Skip this line or handle error as needed
            actual_files[cid] = files

    # Read predicted files and scores from .teIn file
    predicted_files = {}
    with open(teIn_path, 'r') as f:
        for line in f:
            parts = line.strip().split('\t')
            cid = parts[0]
            file_path = parts[1]
            # score = float(parts[2]) # Score is not used directly in evaluation, but might be useful for other purposes

            if cid not in predicted_files:
                predicted_files[cid] = []
            predicted_files[cid].append(file_path)

    # Evaluate results and write to .teOut file
    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, "output.teOut")

    with open(output_path, 'w') as f:
        for cid, files_predicted in predicted_files.items():
            actual_files_modified = actual_files.get(cid, [])
            evaluation_results = evaluator.evaluate(files_predicted, actual_files_modified)
            total_list.append(evaluation_results)

            f.write(f"{cid}\t{json.dumps(evaluation_results)}\n")

        # Print average metrics
        f.write(f"Average metrics: {json.dumps(get_avg_metrics(total_list))}\n")

In [14]:
from eval import evaluate_results
evaluate_results('out/facebook_react/bert_reranker/test_out/commit/output.teIn', 'out/facebook_react/bert_reranker/test_out/commit/output.qry', 'out/facebook_react/bert_reranker/test_out/commit/output.teOut', evaluator)

In [ ]:
evaluate_results(f"{bm25_output_path}/output.teIn", f"{bm25_output_path}/output.qry", bm25_output_path)

In [ ]:
# function to take .pkl file as input and return pandas dataframe
def load_pkl(file_path):
    return pd.read_pickle(file_path)

In [ ]:
df = pd.read_parquet('cache/facebook_react/code_reranker/fb_code_df.parquet')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6631 entries, 0 to 6630
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   train_commit_id           6631 non-null   object
 1   train_query               6631 non-null   object
 2   train_original_message    6631 non-null   object
 3   SR_file_path              6631 non-null   object
 4   SR_commit_id              6631 non-null   object
 5   SR_file_content           6631 non-null   object
 6   SR_previous_file_content  6631 non-null   object
 7   SR_diff                   6446 non-null   object
 8   label                     6631 non-null   int64 
dtypes: int64(1), object(8)
memory usage: 466.4+ KB


In [ ]:
# def tmp():
#     args = Args(index_path='data/2_7/facebook_react/index_commit_tokenized', data_path='data/2_7/facebook_react', k=1000, n=100, model_path='microsoft/codebert-base', overwrite_cache=False, batch_size=32, num_epochs=8, learning_rate=1e-05, run_name='e1', num_positives=10, num_negatives=10, train_depth=1000, num_workers=8, train_commits=2000, psg_cnt=5, aggregation_strategy='sump', use_gpu=True, rerank_depth=250, do_train=False, do_eval=True, eval_gold=True, openai_model='gpt4', overwrite_eval=False, sanity_check=False, debug=False, do_combined_train=False, repo_paths=None, best_model_path='data/combined_commit_train/best_model', ignore_gold_in_training=False, use_gpt_train=False, eval_folder='e1', notes='4x (non-commit) bert train mode classifciation, train data = fb only', train_mode='regression', triplet_cache_path=None)
#     from BERTReranker import main
#     main(args)

# tmp()